# Chapter 2 Tutorial: Understanding LLMs and Pre-training

In this tutorial, we will explore the mechanics of LLM architectures, with an emphasis on the differences between masked models and causal models. In the first section, we'll examine some existing pretrained models to understand how they produce their outputs. Once we've demonstrated how LLM's are able to do what they do, we will then run an abbreviated training loop to provide a glimpse into the training process.

## Installation and Imports

In [1]:
!pip install datasets transformers[sentencepiece,torch]
!pip install apache_beam

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.60.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
  Using cached dill-0.3.1.1-py3-none-any.whl
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.16 requires dill>=0.3.8, but you have dill 0.3.1.1 which is incompatible.


In [2]:
import torch
from datasets import load_dataset, DatasetDict

from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    AutoConfig,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)

/home/maruanottoni/miniconda3/envs/nn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Understanding Masked LM's

In [3]:
## The first model we will look at is BERT, which is trained with masked tokens. As an example,
## the text below masks the word "box" from a well-known movie quote.

text = "Life is like a [MASK] of chocolates."

In [4]:
## We'll now see how BERT is able to predict the missing word. We can use HuggingFace to load
## a copy of the pretrained model and tokenizer.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [5]:
## Next, we'll feed our example text into the tokenizer.

encoded_input = tokenizer(text, return_tensors='pt')
print('input_ids:', encoded_input['input_ids'])
print('attention_mask:', encoded_input['attention_mask'])

input_ids: tensor([[ 101, 2166, 2003, 2066, 1037,  103, 1997, 7967, 2015, 1012,  102]])
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [6]:
## input_ids represents the tokenized output. Each integer can be mapped back to the corresponding string.

print(tokenizer.decode([7967]))

chocolate


In [7]:
## The model will then receive the output of the tokenizer. We can look at the BERT model to see exactly how
## it was constructed and what the outputs will be like.

model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [8]:
## The model starts with an embedding of each of the 30,522 possible tokens into 768 dimensions, which at this
## point is simply a representation of each token without any additional information about their relationships
## to one another in the text. Then the encoder attention blocks are applied, updating the embeddings such that
## they now encode each token's contribution to the chunk of text and interactions with other tokens. Notably,
## this includes the masked tokens as well. The final stage is the language model head, which takes the embeddings
## from the masked positions back to 30,522 dimensions. Each index of this final vector corresponds to the
## probability that the token in that position would be the correct choice to fill the mask.


model_output = model(**encoded_input)
output = model_output["logits"]

print(output.shape)

torch.Size([1, 11, 30522])


In [11]:
tokens = encoded_input['input_ids'][0].tolist()
masked_index = tokens.index(tokenizer.mask_token_id)
logits = output[0, masked_index, :]

print(logits.shape)

torch.Size([30522])


In [12]:
probs = logits.softmax(dim=-1)
values, predictions = probs.topk(5)
sequence = tokenizer.decode(predictions)

print('Top 5 predictions:', sequence)
print(values)

Top 5 predictions: box bag bowl jar cup
tensor([0.1764, 0.1688, 0.0419, 0.0336, 0.0262], grad_fn=<TopkBackward0>)


Printing the top 5 predictions and their respective scores, we see that BERT accurately chooses "box" as the most likely replacement for the mask token.

## Understanding Causal LM's

In [13]:
## We now repeat a similar exercise with the causal LLM GPT-2. This model generates
## text following an input, instead of replacing a mask within the text.

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [14]:
## We can examine the model again, noting the similarities to BERT. An embedding, 12 attention blocks,
## and a linear transformation bringing the output back to the size of the tokenizer. The tokenizer is
## different from BERT so we see we have more tokens this time.

model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [15]:
## We'll use a different text example, since this model works by producing tokens sequentially
## rather than filling a mask.

text = "Swimming at the beach is"
model_inputs = tokenizer(text, return_tensors='pt')
model_inputs

{'input_ids': tensor([[10462, 27428,   379,   262, 10481,   318]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [16]:
## After applying the model, the information needed to predict the next token is represented by
## the last token. So we can access that vector by the index -1.

output = model(**model_inputs)
next_token_logits = output.logits[:, -1, :]
next_token = torch.argmax(next_token_logits, dim=-1)
print(next_token)

tensor([257])


In [17]:
## Now add the new token to the end of the text, and feed all of it back to the model to continue
## predicting more tokens.

model_inputs['input_ids'] = torch.cat([model_inputs['input_ids'], next_token[:, None]], dim=-1)
model_inputs["attention_mask"] = torch.cat([model_inputs['attention_mask'], torch.tensor([[1]])], dim=-1)
print(model_inputs)

{'input_ids': tensor([[10462, 27428,   379,   262, 10481,   318,   257]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [18]:
## Here's what we have so far. The model added the word 'a' to the input text.

print(tokenizer.decode(model_inputs['input_ids'][0]))

Swimming at the beach is a


In [19]:
## Repeating all the previous steps, we then add the word 'great'.

output = model(**model_inputs)
next_token_logits = output.logits[:, -1, :]
next_token = torch.argmax(next_token_logits, dim=-1)
model_inputs['input_ids'] = torch.cat([model_inputs['input_ids'], next_token[:, None]], dim=-1)
model_inputs["attention_mask"] = torch.cat([model_inputs['attention_mask'], torch.tensor([[1]])], dim=-1)
print(tokenizer.decode(model_inputs['input_ids'][0]))

Swimming at the beach is a great


In [20]:
## HuggingFace automates this iterative process. We'll use the quicker approach to finish our sentence.

output_generate = model.generate(**model_inputs, max_length=20, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_generate[0]))

Swimming at the beach is a great way to get a little extra energy.

The beach


## Pre-training a GPT-2 model from scratch

Next we'll train a GPT-2 model from scratch using English Wikipedia data. Note that we're only using a tiny subset of the data to demonstrate that the model is capable of learning. The exact same approach could be followed on the full dataset to train a more functional model, but that would require a lot of compute.

In [21]:
dataset = load_dataset("wikipedia", "20220301.en")
ds_shuffle = dataset['train'].shuffle()

raw_datasets = DatasetDict(
    {
        "train": ds_shuffle.select(range(50)),
        "valid": ds_shuffle.select(range(50, 100))
    }
)

raw_datasets

Generating train split: 100%|██████████| 6458670/6458670 [07:22<00:00, 14591.17 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 50
    })
    valid: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 50
    })
})

In [22]:
print(raw_datasets['train'][0]['text'][:200])

Victoria Serena Pickett (born August 12, 1996) is a Filipino-Canadian soccer player who plays as a midfielder for Kansas City Current in the National Women's Soccer League (NWSL) and the Canada nation


In [23]:
## We'll tokenize the text, setting the context size to 128 and thus breaking each document into chunks of 128 tokens.

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("gpt2")

outputs = tokenizer(
    raw_datasets["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 7
Input chunk lengths: [128, 128, 128, 103, 128, 128, 43]
Chunk mapping: [0, 0, 0, 0, 1, 1, 1]


In [24]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

Map: 100%|██████████| 50/50 [00:00<00:00, 244.39 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 182
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 292
    })
})

Now we can set up the HuggingFace Trainer as follows. Since we're using such a small dataset, we'll need lots of epochs for the model to make progress because all of the parameters are randomly initialized at the outset. Typically, most LLM's are trained for only one epoch and more diverse examples.

In [25]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)

In [26]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [27]:
args = TrainingArguments(
    output_dir="wiki-gpt2",
    evaluation_strategy="steps",
    num_train_epochs=100
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"]
)

/home/maruanottoni/miniconda3/envs/nn/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/maruanottoni/miniconda3/envs/nn/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/tmp/ipykernel_18127/148413081.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = T

In [28]:
trainer.train()

Step,Training Loss,Validation Loss
500,4.521200,8.916580
1000,0.964600,9.287793


TrainOutput(global_step=1200, training_loss=2.340398515065511, metrics={'train_runtime': 8230.6977, 'train_samples_per_second': 2.211, 'train_steps_per_second': 0.146, 'total_flos': 1188878745600000.0, 'train_loss': 2.340398515065511, 'epoch': 100.0})

In [29]:
trainer.evaluate()

{'eval_loss': 9.30921745300293,
 'eval_runtime': 51.0598,
 'eval_samples_per_second': 5.719,
 'eval_steps_per_second': 0.372,
 'epoch': 100.0}

The training loss is low by the end, which means the model should perform very well on training examples it has seen. It does not generalize well to the validation set of course, since we deliberately overfit on a small train set.

We can confirm with a couple of examples that were seen in training.

In [30]:
text = tokenizer.decode(tokenized_datasets["train"][0]['input_ids'][:16])
print(text)

Victoria Serena Pickett (born August 12, 1996) is a Filipino-


In [31]:
model_inputs = tokenizer(text, return_tensors='pt')
print(model_inputs['input_ids'].shape)

torch.Size([1, 16])


In [32]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_inputs['input_ids'] = model_inputs['input_ids'].to(device)
model_inputs['attention_mask'] = model_inputs['attention_mask'].to(device)

output_generate = model.generate(**model_inputs, max_new_tokens=16)
output_generate

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[49898, 30175,  2616, 12346,  3087,   357,  6286,  2932,  1105,    11,
          8235,     8,   318,   257, 35289,    12, 28203, 11783,  2137,   508,
          5341,   355,   257, 18707,   329,  9470,  2254,  9236,   287,   262,
          2351,  6926]])

In [33]:
sequence = tokenizer.decode(output_generate[0])
print(sequence)

Victoria Serena Pickett (born August 12, 1996) is a Filipino-Canadian soccer player who plays as a midfielder for Kansas City Current in the National Women


The model should do quite well at reciting text after seeing it so many times. We can be convinced that the tokenizer, model architecture, and training objective are well-suited to learning Wikipedia data. For comparison, we'll try this model on text from the validation set.

In [34]:
text = tokenizer.decode(tokenized_datasets["valid"][0]['input_ids'][:32])
print(text)

Charles II (29 May 1630 – 6 February 1685) was King of Scotland from 1649 until 1651, and King of Scotland, England and Ireland


In [35]:
model_inputs = tokenizer(text, return_tensors='pt')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_inputs['input_ids'] = model_inputs['input_ids'].to(device)
model_inputs['attention_mask'] = model_inputs['attention_mask'].to(device)

output_generate = model.generate(**model_inputs, max_new_tokens=16)
sequence = tokenizer.decode(output_generate[0])
print(sequence)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Charles II (29 May 1630 – 6 February 1685) was King of Scotland from 1649 until 1651, and King of Scotland, England and Ireland also begun suffering with injury problems, also facing stiff left-back competition from younger


In [36]:
raw_datasets['valid'][0]['text']

'Charles II (29 May 1630 – 6 February 1685) was King of Scotland from 1649 until 1651, and King of Scotland, England and Ireland from the 1660 Restoration of the monarchy until his death in 1685.\n\nCharles II was the eldest surviving child of Charles I of England, Scotland and Ireland and Henrietta Maria of France. After Charles I\'s execution at Whitehall on 30 January 1649, at the climax of the English Civil War, the Parliament of Scotland proclaimed Charles II king on 5 February 1649. But England  entered the period known as the English Interregnum or the English Commonwealth, and the country was a de facto republic led by Oliver Cromwell. Cromwell defeated Charles II at the Battle of Worcester on 3 September 1651, and Charles fled to mainland Europe. Cromwell became virtual dictator of England, Scotland and Ireland. Charles spent the next nine years in exile in France, the Dutch Republic and the Spanish Netherlands. The political crisis that followed Cromwell\'s death in 1658 resu

As expected, our model is completely confused this time. We'd need to train for much longer, and on much more diverse data, before we would have a model that can sensibly complete prompts it has never seen before. This is precisely why pre-training is such an important and powerful technique. If we had to train on all of Wikipedia for every NLP application to achieve optimal performance, it would be prohibitively expensive. But there's no need to do that when we can share and reuse existing pre-trained models as we did in the first part of this tutorial.